In [65]:
# !pip install pysmiles
# !pip install pyTDC
print('hello world')

hello world


In [66]:
import networkx as nx
import numpy as np
import pandas as pd
from pysmiles import read_smiles

# from tdc.single_pred import ADME
import copy

# Psudo Code

Color refinement with different starting colors

There are a couple of things to keep in mind. First, You want to keep the number of unique colors limited. 

If you use somethign like:

Node color (i+1) =  hash (node color (i) ~append? multiply? add? (color for color in Node neighbors) % N You are garenteed to have at most N colors. Where you run into problems when mutiplue colors are hashed together. 

Odds of a spurious hash for a single node == 1/N

Odds of a spurious hash for a single iteration == (num nodes) / N

Odds of a spurious hash for k iteratiosn = K *(num_nodes) / N

### I have not double checked this math. I should run it by someone who knows it better. You should run some simulations on this.


Requirements


1. Given a graph where each vertex is garenteed to have exactly one attribute that can be treated as a int. 





In [67]:
g = read_smiles('COc1ccccc1CNC(=O)Cn1nnc(-c2ccncc2)n1')
g

### Next steps:


1. Empericla test your functions at differetn number of buckets to make sure that they are evenly distributed.
2. Wrap the hashing functions in factory classes
3. Make the color iterators do `color_0` for the inital coloring and then `color_1` ... `color_k` for the iterations after than that. (the graph features when will each be a num_buckets long feature K features for K iterations.


# You need a emperical measure of colisions.


In [111]:
import numpy as np
import pandas as pd
import networkx as nx


class ColorRefinement:

  @staticmethod
  def create_color_hash_function(num_buckets: int):

    def get_next_node_color(node_color:int, neighbor_colors:list)-> int:
      pre_hash_color = str(node_color) + str(np.prod(neighbor_colors))
      new_color = hash(pre_hash_color) % num_buckets
      return new_color


    hash_function = get_next_node_color
    return hash_function

  @staticmethod
  def convert_node_view_to_node_color_dict(node_data: nx.classes.reportviews.NodeDataView, num_buckets: int)-> dict:
    """
        Set the inital color of each of the nodes based on hashing the node attributes and a number of buckets. 

        Returns a dictionary of inital colors to make the nodes
    """

    node_data_as_dict = dict(node_data)
    node_names = list(node_data_as_dict.keys())
    node_starting_colors = dict()

    for index, node_num in enumerate(node_names):
      node_attributes = tuple(sorted(list(node_data_as_dict[node_names[index]].items())))
      inital_node_color = hash(node_attributes) % num_buckets
      node_starting_colors[node_num] = inital_node_color

    return node_starting_colors
  
  @staticmethod
  def create_inital_color_graph(graph: nx.classes.graph.Graph, num_buckets:int) ->nx.classes.graph.Graph:

    starting_colors = ColorRefinement.convert_node_view_to_node_color_dict(graph.nodes.data(True), num_buckets)
    color_graph = copy.deepcopy(graph)
    nx.set_node_attributes(color_graph, starting_colors, name="color_0")

    # remove unneeded tags
    for index, node in enumerate(color_graph.nodes.data(True)):
      node_attributes = list(color_graph.nodes[index].keys())
      attributes_to_remove = [a for a in node_attributes if a !='color_0']
      [color_graph.nodes[index].pop(a) for a in attributes_to_remove]

    return color_graph

  @staticmethod
  def next_iteration_of_color_graph(color_graph, iter_num:int, color_hashing_function) -> None:
    prev_color_key = f'color_{iter_num-1}'
    
    # do an iteration
    updated_colors = dict()
    for index, current_node in enumerate(color_graph.nodes.data(True)):
      neighbor_nodes = [n for n in color_graph.neighbors(index)]
      current_color = color_graph.nodes[current_node[0]]
      neighbor_colors = ([color_graph.nodes[n][prev_color_key] for n in neighbor_nodes])
      updated_colors[index] = color_hashing_function(current_color,neighbor_colors)

    nx.set_node_attributes(color_graph, updated_colors, name=f'color_{iter_num}')

  @staticmethod
  def compute_K_color_refinements(G: nx.classes.graph.Graph, K:int, num_buckets:int) ->nx.classes.graph.Graph:
    """
    Given a graph G and a hash_function f, compute K iterations of the color refinement algorithm (link to algo)
    """
    color_hashing_function = ColorRefinement.create_color_hash_function(num_buckets)
    color_graph = ColorRefinement.create_inital_color_graph(G, num_buckets)

    for iter_num in range(1,K):
      ColorRefinement.next_iteration_of_color_graph(color_graph, iter_num=iter_num, color_hashing_function=color_hashing_function)
    return color_graph


  @staticmethod
  def extact_node_colors(node, node_num)-> tuple:
    df = pd.DataFrame.from_dict(node.items())
    df.columns = ['iteration_number', f'Node_{node_num}']
    df.set_index('iteration_number', inplace=True)
    return df


  @staticmethod
  def convert_color_graph_to_DataFrame(color_graph: nx.classes.graph.Graph) -> pd.DataFrame:
    """
      Converts the node colors into a pandas data frame where the rows are the iterations and the columns are the node colors
    """
    color_node_view = color_graph.nodes.data(True)
    node_colors = [ColorRefinement.extact_node_colors(color_node_view[i], i) for i in range(0,len(color_node_view))]
    df = pd.concat(node_colors, axis=1)
    return df

  @staticmethod
  def compute_graph_embeddings(G: nx.classes.graph.Graph, K:int, num_buckets:int) -> pd.DataFrame:
    color_graph = ColorRefinement.compute_K_color_refinements(G=G, K=K, num_buckets=num_buckets)
    embedding_df = ColorRefinement.convert_color_graph_to_DataFrame(color_graph=color_graph)
    return embedding_df

  @staticmethod
  def compute_bag_of_colors_vector(iteration_colors: np.array, num_buckets:int):
    bag_of_colors = np.zeros(shape=num_buckets).astype(int)
    for color in iteration_colors:
        bag_of_colors[color] +=1
    return bag_of_colors


  @staticmethod
  def compute_call_color_bag(embedding_df: pd.DataFrame, num_buckets:int) -> pd.DataFrame:
    bag_of_colors_df = pd.DataFrame(index= embedding_df.index, columns=[a for a in range(num_buckets)])
    for iteration_name in bag_of_colors_df.index:
       iteration_colors = embedding_df.loc[iteration_name].values
       bag_of_colors_vector = ColorRefinement.compute_bag_of_colors_vector(iteration_colors, num_buckets)
       bag_of_colors_df.loc[iteration_name, :] = bag_of_colors_vector

    return bag_of_colors_df

  @staticmethod
  def embedd_graph_with_color_refinement(G: nx.classes.graph.Graph, K:int, num_buckets:int) -> pd.DataFrame:
    embedding_df = ColorRefinement.compute_graph_embeddings(G,K,num_buckets)
    bag_of_colors_df = ColorRefinement.compute_call_color_bag(embedding_df,num_buckets)
    return bag_of_colors_df

cr = ColorRefinement()

df = cr.embedd_graph_with_color_refinement(g,11,10_000)
df.sum(axis=1)

Index(['color_0', 'color_1', 'color_2', 'color_3', 'color_4', 'color_5',
       'color_6', 'color_7', 'color_8', 'color_9', 'color_10'],
      dtype='object', name='iteration_number')


iteration_number
color_0     24.0
color_1     24.0
color_2     24.0
color_3     24.0
color_4     24.0
color_5     24.0
color_6     24.0
color_7     24.0
color_8     24.0
color_9     24.0
color_10    24.0
dtype: float64

In [89]:
for iteration_name in df.index:
    print(type(df.loc[iteration_name].values))



<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [92]:
s = np.array([3,4,4,8])
e = np.zeros(10).astype(np.int)

for i in s:
    e[i] +=1
e

array([0, 0, 0, 1, 2, 0, 0, 0, 1, 0])

In [93]:
def compute_bag_of_colors_vector(iteration_colors: np.array, num_buckets:int):
    bag_of_colors = np.zeros(shape=num_buckets).astype(int)

    for color in iteration_colors:
        bag_of_colors[color] +=1
    return bag_of_colors
